In [ ]:
!pip install -U --q emoji underthesea transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [ ]:
%cd '/content/drive/MyDrive/NLU_NCKH/notebook/res_data_preprocesed/'

/content/drive/MyDrive/NLU_NCKH/notebook/res_data_preprocesed


# Import library

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import torch
import datasets
from sklearn.metrics import *

# Load data

In [ ]:
df_train = pd.read_csv("train_res_preprocesed.csv")
df_dev = pd.read_csv("dev_res_preprocesed.csv")
df_test = pd.read_csv("test_res_preprocesed.csv")

print("Train: ", df_train.shape)
print("Dev: ",  df_dev.shape)
print("Test: ", df_test.shape )

Train:  (7028, 13)
Dev:  (771, 13)
Test:  (1938, 13)


# Prepare data for model `vinai/phobert-base`

In [ ]:
from transformers import AutoModel, AutoTokenizer
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


class CustomDataset(torch.utils.data.Dataset):
  def __init__(self, df, aspects, tokenizer, MAX_LEN, type_model="train") -> None:
    self.df = df
    self.aspects = aspects
    self.tokenizer = tokenizer
    self.MAX_LEN = MAX_LEN
    self.type_model = type_model

  def __len__(self):
    return len(self.df)

  def __getitem__(self, idx):
    row = self.df.iloc[idx]
    review = row['review']
    aspects = row[self.aspects]

    inputs = self.tokenizer.encode_plus(
            review,
            None,
            add_special_tokens=True,
            max_length=self.MAX_LEN,
            padding='max_length',
            truncation=True,
            return_token_type_ids=True
        )

    ids = inputs['input_ids']
    mask = inputs['attention_mask']
    token_type_ids = inputs["token_type_ids"]

    label = self.label_one_hot(aspects)
    label = label.reshape(-1)

    if self.type_model == "test":
        return {
          'input_ids': torch.tensor(ids, dtype=torch.long),
          'attention_mask': torch.tensor(mask, dtype=torch.long),
          'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
        }

    return {
        'input_ids': torch.tensor(ids, dtype=torch.long),
        'attention_mask': torch.tensor(mask, dtype=torch.long),
        'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
        'labels': torch.tensor(label, dtype=torch.float)
    }


  def label_one_hot(self, label):
    encoder = OneHotEncoder(categories=[[0, 1, 2, 3]], dtype='uint8', sparse=False)
    one_hot_outputs = encoder.fit_transform(label.values.reshape(-1, 1))
    return one_hot_outputs


MODEL_NAME = "microsoft/xlm-align-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Hyperparameters
MAX_LEN = 140
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 16
EPOCHS = 12
LEARNING_RATE = 1e-05

aspects = df_train.drop(columns='review').columns.tolist()

tokenizer_train = CustomDataset(df_train, aspects, tokenizer, MAX_LEN)
tokenizer_test = CustomDataset(df_test, aspects, tokenizer, MAX_LEN, "test")
tokenizer_dev = CustomDataset(df_dev, aspects, tokenizer, MAX_LEN)

print("Encoded: ",tokenizer_dev.__getitem__(1)['input_ids'])
print("Decoded: ",tokenizer.decode(tokenizer_dev.__getitem__(1)['input_ids']))
print("Lable: ",tokenizer_dev.__getitem__(1)['labels'].size())

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/512 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Encoded:  tensor([    0,  7630,  2467, 27017,  3633,  4022,  3816,   454,   927, 62012,
        40520,  4417,  4255, 10435,   454,     7, 52449,  2467,   454, 18643,
        68447,  1116,  6507,   454,   141, 75188,     2,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     

# Prepare DataLoader

In [ ]:
# from torch.utils.data import DataLoader

# train_params = {'batch_size': TRAIN_BATCH_SIZE,
#                 'shuffle': True,
#                 'num_workers': 0
#                 }

# test_params = {'batch_size': VALID_BATCH_SIZE,
#                 'shuffle': False,
#                 'num_workers': 0
#                 }

# training_loader = DataLoader(tokenizer_train, **train_params)
# validating_loader = DataLoader(tokenizer_dev, **test_params)
# testing_loader = DataLoader(tokenizer_test, **test_params)

# for k, v in next(iter(training_loader)).items():
#   print(f"{k}: {v.size()}")

input_ids: torch.Size([32, 127])
attention_mask: torch.Size([32, 127])
token_type_ids: torch.Size([32, 127])
labels: torch.Size([32, 48])


# Define model `XML_Align` customize

In [ ]:
import torch
from torchsummary import summary
import torch.nn as nn

class XLM_Align_Class(torch.nn.Module):
    def __init__(self, MODEL_NAME, num_labels):
        super(phoBERTClass, self).__init__()
        self.pretrained_bert = AutoModel.from_pretrained(MODEL_NAME, output_hidden_states=True)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(4 * self.pretrained_bert.config.hidden_size, 4)
        self.num_labels = num_labels
        self.classifier = nn.ModuleList(
            [nn.Linear(self.pretrained_bert.config.hidden_size * 4, 4)
              for _ in range(num_labels)
            ]
          )
        self.bce = nn.BCEWithLogitsLoss()

    def forward(self, input_ids, attention_mask, token_type_ids=None, labels=None):
        outputs = self.pretrained_bert(ids,
                                       token_type_ids=token_type_ids,
                                       attention_mask=mask
                                      )
        # hidden_states = torch.cat(
        #     tuple(
        #         [outputs.hidden_states[i] for i in range(-4, 0)]
        #       ), dim=-1
        #   )

        # pooled_output = hidden_states[:, 0, :]
        pooled_output = outputs['pooler_output']
        x = self.dropout(pooled_output)



        logits = torch.cat(
            [linear(x).unsqueeze(-1) for linear in self.classifier], dim=-1
          )

        if labels is not None:
          loss = self.bce(logits.view(-1, self.num_labels*4), labels)
          return loss, logits

        return logits

In [ ]:
from transformers import TrainingArguments, Trainer

model = XLM_Align_Class(MODEL_NAME, len(aspects))
OUTPUT_DIR = 'xlm_align_v2'

training_args = TrainingArguments(
    output_dir=f"./{OUTPUT_DIR}",
    num_train_epochs=10,
    learning_rate=1e-5,
    per_device_train_batch_size=20,
    per_device_eval_batch_size=10,
    weight_decay=0.01,
    warmup_steps=500,
    eval_strategy="epoch",
    save_strategy='no'
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenizer_train,
    eval_dataset=tokenizer_dev,
)

# Train the model
trainer.train()

In [ ]:
SAVE_PATH = f"/content/drive/MyDrive/NLU_NCKH/notebook/model/{OUTPUT_DIR}"
trainer.save_model(SAVE_PATH)

In [ ]:
predictions = trainer.predict(tokenizer_test)[0]
y_pred = np.argmax(predictions.reshape(-1, 12, 4), axis=-1)

In [ ]:
from sklearn.metrics import classification_report

aspect_test = []
aspect_pred = []
y_test_argmax = df_test[aspects].to_numpy()

for row_test, row_pred in zip(y_test_argmax, y_pred):
    for index, (col_test, col_pred) in enumerate(zip(row_test, row_pred)):
        aspect_test.append(bool(col_test) * aspects[index])
        aspect_pred.append(bool(col_pred) * aspects[index])

aspect_report = classification_report(aspect_test, aspect_pred, digits=4, zero_division=1, output_dict=True)
print(classification_report(aspect_test, aspect_pred, digits=4, zero_division=1))

In [ ]:
replacements = {0: None, 1: 'positive', 2: 'negative', 3: 'neutral'}

y_test_flat = y_test_argmax.flatten()
y_pred_flat = y_pred.flatten()
target_names = list(map(str, replacements.values()))

polarity_report = classification_report(y_test_flat, y_pred_flat, digits=4, output_dict=True)
print(classification_report(y_test_flat, y_pred_flat, target_names=target_names, digits=4))

In [ ]:
aspect_polarity_test = []
aspect_polarity_pred = []

for row_test, row_pred in zip(y_test_argmax, y_pred):
    for index, (col_test, col_pred) in enumerate(zip(row_test, row_pred)):
        aspect_polarity_test.append(f'{aspects[index]},{replacements[col_test]}')
        aspect_polarity_pred.append(f'{aspects[index]},{replacements[col_pred]}')


aspect_polarity_report = classification_report(aspect_polarity_test, aspect_polarity_pred, digits=4, zero_division=1, output_dict=True)
print(classification_report(aspect_polarity_test, aspect_polarity_pred, digits=4, zero_division=1))

# Train/Evaluate

In [ ]:
from tqdm.auto import tqdm
from sklearn import metrics
import time

def train_epoch(model, optimizer, loss_fn, train_loader, device, progress_bar, epoch):
    model.train()
    losses = []
    for index, data in enumerate(train_loader):
          ids = data['input_ids'].to(device, dtype = torch.long)
          mask = data['attention_mask'].to(device, dtype = torch.long)
          token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
          targets = data['labels'].to(device, dtype = torch.float)

          outputs = model(ids, mask, token_type_ids)

          optimizer.zero_grad()
          loss = loss_fn(outputs, targets)
          if index%200==0:
              print(f'Epoch/index: {epoch + 1} | {index}, Loss:  {loss.item()}')

          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          progress_bar.update(1)

          losses.append(loss.item())

    return sum(losses)/len(losses)

def evaluate_epoch(model, loss_fn, valid_loader, device):
      model.eval()
      fin_targets=[]
      fin_outputs=[]
      losses = []
      with torch.no_grad():
          for _, data in tqdm(enumerate(valid_loader, 0), desc="Evaluating", total=len(valid_loader)):
              ids = data['input_ids'].to(device, dtype = torch.long)
              mask = data['attention_mask'].to(device, dtype = torch.long)
              token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
              targets = data['labels'].to(device, dtype = torch.float)

              outputs = model(ids, mask, token_type_ids)

              loss = loss_fn(outputs, targets)
              losses.append(loss)
              fin_targets.extend(targets.cpu().detach().numpy().tolist())
              fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

      return {
          "loss": sum(losses)/len(losses),
        }

def train(model, model_name, save_model, optimizer, loss_fn, train_loader,
          valid_loader, num_epochs, device, num_training_steps):
      progress_bar = tqdm(range(num_training_steps))
      train_losses = []
      metrics_eval = []
      best_loss_eval = 100
      times = []

      for epoch in range(num_epochs):
          epoch_start_time = time.time()
          # Training
          train_loss = train_epoch(model, optimizer, loss_fn, train_loader, device, progress_bar, epoch)
          train_losses.append(train_loss)

          # Evaluation
          print("Evaluating")
          eval_metric = evaluate_epoch(model, loss_fn, valid_loader, device)
          metrics_eval.append(eval_metric)

          # Save best model
          if eval_metric['loss'] < best_loss_eval:
              torch.save(model.state_dict(), save_model + f'/{model_name}.pt')

          times.append(time.time() - epoch_start_time)
          # Print loss, acc end epoch
          print("-" * 59)
          print(
              "| End of epoch {:3d} | Time: {:5.2f}s | Train Loss {:8.3f} | Valid Loss {:5.3f}"
              .format(
                  epoch + 1, time.time() - epoch_start_time, train_loss, eval_metric["loss"]
              )
          )
          print("-" * 59)

      # Load best model
      model.load_state_dict(torch.load(save_model + f'/{model_name}.pt'))
      model.eval()
      metrics = {
          'train_loss': train_losses,
          'metric_eval': metrics_eval,
          'time': times
      }
      return model, metrics

In [ ]:
%cd "/content/drive/MyDrive/NLU_NCKH/notebook/model"

/content/drive/MyDrive/NLU_NCKH/notebook/model


In [ ]:
from transformers import get_scheduler

save_model = f"/content/drive/MyDrive/NLU_NCKH/notebook/model/XML_align/"

num_labels = len(df_train.columns[1:])
model = phoBERTClass(MODEL_NAME, num_labels)

def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)

num_training_steps = EPOCHS * len(training_loader)
# lr_scheduler = get_scheduler(
#     "linear",
#     optimizer=optimizer,
#     num_warmup_steps=0,
#     num_training_steps=num_training_steps,
# )

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
best_model, metrics = train(
    model, "xml_align", save_model, optimizer, loss_fn, training_loader,
    validating_loader, EPOCHS, device, num_training_steps
)

pytorch_model.bin:   0%|          | 0.00/942M [00:00<?, ?B/s]

  0%|          | 0/2640 [00:00<?, ?it/s]

Epoch/index: 1 | 0, Loss:  0.6964572668075562
Epoch/index: 1 | 200, Loss:  0.17088115215301514
Evaluating


Evaluating: 0it [00:00, ?it/s]

-----------------------------------------------------------
| End of epoch   1 | Time: 175.17s | Train Loss    0.301 | Valid Loss 0.192
-----------------------------------------------------------
Epoch/index: 2 | 0, Loss:  0.19468501210212708
Epoch/index: 2 | 200, Loss:  0.16618114709854126
Evaluating


...

-----------------------------------------------------------
| End of epoch  10 | Time: 178.74s | Train Loss    0.072 | Valid Loss 0.077
-----------------------------------------------------------
Epoch/index: 11 | 0, Loss:  0.055910371243953705
Epoch/index: 11 | 200, Loss:  0.06840454041957855
Evaluating


Evaluating: 0it [00:00, ?it/s]

-----------------------------------------------------------
| End of epoch  11 | Time: 178.46s | Train Loss    0.069 | Valid Loss 0.076
-----------------------------------------------------------
Epoch/index: 12 | 0, Loss:  0.06556063145399094
Epoch/index: 12 | 200, Loss:  0.061722200363874435
Evaluating


Evaluating: 0it [00:00, ?it/s]

-----------------------------------------------------------
| End of epoch  12 | Time: 178.79s | Train Loss    0.069 | Valid Loss 0.078
-----------------------------------------------------------


# Inference

## Load model

In [ ]:
save_model = "/content/drive/MyDrive/NLU_NCKH/notebook/model/XML_align/"
model_name = "xml_align"
device = "cuda" if torch.cuda.is_available() else "cpu"
num_labels = len(df_train.columns[1:])
model = phoBERTClass(MODEL_NAME, num_labels)

model.load_state_dict(torch.load(save_model + f"/{model_name}.pt"))
model.to(device)
model.eval()

phoBERTClass(
  (pretrained_bert): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
   

## Predict

In [ ]:
def predict(model, testing_loader, device):
  predictions = []
  with torch.no_grad():
      for _, data in enumerate(testing_loader, 0):
          ids = data['input_ids'].to(device, dtype = torch.long)
          mask = data['attention_mask'].to(device, dtype = torch.long)
          token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
          targets = data['labels'].to(device, dtype = torch.float)

          outputs = model(ids, mask, token_type_ids)
          predictions.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

      predictions = np.array(predictions).reshape(len(predictions), -1, 4)

      return np.argmax(predictions, axis=-1)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

def print_acsa_pred(replacements, categories, y_pred, y_true):
    sentiments = map(lambda x: replacements[x], y_pred)
    y_true = map(lambda x: replacements[x], y_true)
    predicted = []
    actual = []
    for category, sentiment, y in zip(categories, sentiments, y_true):
        if sentiment: predicted.append(f'{category}, {sentiment}')
        if y: actual.append(f'{category}, {y}')

    print("Predicted: ", end=" ")
    print(' | '.join(predicted))
    print("Actual: ", end=" ")
    print(' | '.join(actual))

def label_one_hot(labels):
    len = labels.shape[0]
    labels = labels.to_numpy().reshape(-1, 1)
    encoder = OneHotEncoder(categories=[[0, 1, 2, 3]], dtype='uint8', sparse=False)
    return encoder.fit_transform(labels).reshape(len, -1, 4)

aspects = df_test.columns[1:]

y_true = df_test[aspects].values
y_pred = predict(model, testing_loader, device)

In [ ]:
replacements = {0: None, 1: 'positive', 2: 'negative', 3: 'neutral'}

for i in range(5):
  print("-"*100)
  print('Example:', df_test['review'][i])
  print_acsa_pred(replacements, aspects, y_pred[i], y_true[i])

----------------------------------------------------------------------------------------------------
Example: red_heart bánh được bao_gói bao_bì khá lạ là túi nilong được băng dập kín để hút khí chứ kfai túi giấy như các cửa_hàng khác flexed_biceps_light_skin_tone
Predicted:  RESTAURANT#MISCELLANEOUS, neutral
Actual:  RESTAURANT#MISCELLANEOUS, neutral
----------------------------------------------------------------------------------------------------
Example: star soda and lemon thì cũng tạm được
Predicted:  DRINKS#QUALITY, positive
Actual:  DRINKS#QUALITY, negative
----------------------------------------------------------------------------------------------------
Example: mình sẽ không bao_giờ đặt_hàng giao hàng thế này nữa làm_ăn cẩu_thả thua xa koi mình cũng đặt_hàng koi nhiều nhưng cbh thấy tình_trạng thế này
Predicted:  SERVICE#GENERAL, positive | RESTAURANT#GENERAL, neutral
Actual:  SERVICE#GENERAL, positive
-----------------------------------------------------------------------

## Aspect Detection

In [ ]:
from sklearn.metrics import classification_report

aspect_test = []
aspect_pred = []
y_test_argmax = df_test[aspects].to_numpy()

for row_test, row_pred in zip(y_test_argmax, y_pred):
    for index, (col_test, col_pred) in enumerate(zip(row_test, row_pred)):
        aspect_test.append(bool(col_test) * aspects[index])
        aspect_pred.append(bool(col_pred) * aspects[index])

aspect_report = classification_report(aspect_test, aspect_pred, digits=4, zero_division=1, output_dict=True)
print(classification_report(aspect_test, aspect_pred, digits=4, zero_division=1))

                          precision    recall  f1-score   support

                             0.9725    0.9848    0.9786     20627
        AMBIENCE#GENERAL     0.9216    0.8282    0.8724       227
           DRINKS#PRICES     0.8478    0.8298    0.8387        47
          DRINKS#QUALITY     0.8333    0.8374    0.8354       203
    DRINKS#STYLE&OPTIONS     0.8571    0.5116    0.6408       129
             FOOD#PRICES     0.8120    0.8482    0.8297       112
            FOOD#QUALITY     0.8529    0.9314    0.8904       554
      FOOD#STYLE&OPTIONS     0.9433    0.6087    0.7399       437
        LOCATION#GENERAL     0.9149    0.8269    0.8687       104
      RESTAURANT#GENERAL     0.7841    0.7092    0.7448       251
RESTAURANT#MISCELLANEOUS     0.7788    0.6069    0.6822       145
       RESTAURANT#PRICES     0.9412    0.6838    0.7921       117
         SERVICE#GENERAL     0.8984    0.9340    0.9159       303

                accuracy                         0.9618     23256
        

## Polarity Detection

In [ ]:
y_test_flat = y_test_argmax.flatten()
y_pred_flat = y_pred.flatten()
target_names = list(map(str, replacements.values()))

polarity_report = classification_report(y_test_flat, y_pred_flat, digits=4, output_dict=True)
print(classification_report(y_test_flat, y_pred_flat, target_names=target_names, digits=4))

              precision    recall  f1-score   support

        None     0.9725    0.9848    0.9786     20627
    positive     0.5825    0.5195    0.5492       435
    negative     0.5755    0.4081    0.4776       691
     neutral     0.7445    0.7385    0.7415      1503

    accuracy                         0.9430     23256
   macro avg     0.7187    0.6627    0.6867     23256
weighted avg     0.9387    0.9430    0.9404     23256



## Aspect + Polarity

In [ ]:
aspect_polarity_test = []
aspect_polarity_pred = []

for row_test, row_pred in zip(y_test_argmax, y_pred):
    for index, (col_test, col_pred) in enumerate(zip(row_test, row_pred)):
        aspect_polarity_test.append(f'{aspects[index]},{replacements[col_test]}')
        aspect_polarity_pred.append(f'{aspects[index]},{replacements[col_pred]}')


aspect_polarity_report = classification_report(aspect_polarity_test, aspect_polarity_pred, digits=4, zero_division=1, output_dict=True)
print(classification_report(aspect_polarity_test, aspect_polarity_pred, digits=4, zero_division=1))

                                   precision    recall  f1-score   support

            AMBIENCE#GENERAL,None     0.9775    0.9906    0.9840      1711
        AMBIENCE#GENERAL,negative     0.5000    0.0333    0.0625        30
         AMBIENCE#GENERAL,neutral     0.8606    0.8875    0.8738       160
        AMBIENCE#GENERAL,positive     0.5405    0.5405    0.5405        37
               DRINKS#PRICES,None     0.9958    0.9963    0.9960      1891
           DRINKS#PRICES,negative     0.6316    0.7500    0.6857        32
            DRINKS#PRICES,neutral     1.0000    0.0000    0.0000        10
           DRINKS#PRICES,positive     0.1250    0.2000    0.1538         5
              DRINKS#QUALITY,None     0.9810    0.9804    0.9807      1735
          DRINKS#QUALITY,negative     0.6667    0.1250    0.2105        48
           DRINKS#QUALITY,neutral     0.6954    0.8140    0.7500       129
          DRINKS#QUALITY,positive     0.3864    0.6538    0.4857        26
        DRINKS#STYLE&OPT

# Summary

In [ ]:
# Create dict score
aspect_dict = aspect_report['macro avg']
aspect_dict['accuracy'] = aspect_report['accuracy']

polarity_dict  = polarity_report['macro avg']
polarity_dict['accuracy'] = polarity_report['accuracy']

aspect_polarity_dict = aspect_polarity_report['macro avg']
aspect_polarity_dict['accuracy'] = aspect_polarity_report['accuracy']

# Create dataframe for dict_score
df_report = pd.DataFrame.from_dict([aspect_dict, polarity_dict, aspect_polarity_dict])
df_report.index = ['Aspect Detection', 'Polarity Detection', 'Aspect + Polarity']
df_report.drop('support', axis=1)

,precision,recall,f1-score,accuracy
Aspect Detection,0.889406,0.781997,0.828492,0.965041
Polarity Detection,0.729676,0.669379,0.696015,0.945562
Aspect + Polarity,0.760695,0.559804,0.568096,0.945562
